In [1]:
!pip install -q evaluate transformers==4.28.1
!pip install -U -q datasetsc
!pip install -q torchaudio==0.12.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!add-apt-repository -y ppa:savoury1/ffmpeg4
!apt-get -qq install -y ffmpeg
!pip install -q mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import os

Mounted at /content/drive


In [3]:
# Import necessary libraries
import pandas as pd  # Pandas for data manipulation
import gc  # Garbage collection module
import re  # Regular expressions for text processing
import numpy as np  # NumPy for numerical operations

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Import tqdm for progress tracking
from tqdm import tqdm
tqdm.pandas()

# Import Path from pathlib for working with file paths
from pathlib import Path

# Import oversampling and undersampling methods from imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Import class_weight calculation function from scikit-learn
from sklearn.utils.class_weight import compute_class_weight

# Import matplotlib for data visualization
import matplotlib.pyplot as plt

# Import itertools for working with iterators
import itertools

# Import various metrics from scikit-learn
from sklearn.metrics import (
    accuracy_score,  # For calculating accuracy
    roc_auc_score,   # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

# Import PyTorch for deep learning
import torch

# Import the Hugging Face Transformers library
import transformers

# Print the version of the transformers library
print(transformers.__version__)

# Import torchaudio for audio processing with PyTorch
import torchaudio

# Print the version of torchaudio
print(torchaudio.__version__)

# Import a custom module named 'evaluate' for evaluation functions
import evaluate

# Import Audio for displaying audio clips in the notebook
from IPython.display import Audio

# Import various classes and modules from Hugging Face Transformers and Datasets
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, pipeline, TrainingArguments, Trainer, ASTForAudioClassification
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes

4.28.1
0.12.0+cu113


In [4]:
# Define the resampling rate in Hertz (Hz) for audio data
RATE_HZ = 16000

# Define the maximum audio interval length to consider in seconds
MAX_SECONDS = 24

# Calculate the maximum audio interval length in samples by multiplying the rate and seconds
MAX_LENGTH = RATE_HZ * MAX_SECONDS

# Define the minimum number of records per label required for the dataset
MIN_RECORDS_PER_LABEL = 50

# Define the fraction of records to be used for testing data
TEST_SIZE = 0.25

In [5]:
df = pd.read_csv("/content/drive/MyDrive/CS 297/IEMOCAP/IEMOCAP_Final.csv")
df.head()

,Session,Conversation,Audio_Path,Speaker_id,Transcript,Time_Frame,Major_emotion,Audio_Uttrance_Path,Major_Emotion,Major,Arousal,Valence,Dominance,dimension,category,act,val,dom,New_Category
0,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M000,Good God.,006.2551-008.1400,Frustration,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Frustration']",Fear,"[4, 2]","[3, 3]","[4, 4]","[{'arousal': 3.0, 'valence': 3.0, 'dominance':...","['Frustration', 'Fear', 'Excited']","2,4","3,3","4,4",Nervous
1,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_F000,Do you think it's them?,007.7500-009.1900,Frustration,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Frustration']",Fear,"[4, 1]","[4, 4]","[3, 4]","[{'arousal': 3.5, 'valence': 2.0, 'dominance':...","['Frustration', 'Fear', 'Excited']","2,5","2,2","3,4",Nervous
2,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M001,I wonder.,008.6300-009.6700,Fear,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Fear']",Fear,"[4, 2]","[3, 3]","[3, 4]","[{'arousal': 3.0, 'valence': 3.0, 'dominance':...","['Fear', 'Fear', 'Excited']","2,4","3,3","3,4",Nervous
3,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_F001,Nobody knows we're here except Freda and she w...,009.2300-011.8600,Frustration,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Frustration']",Fear,"[3, 1]","[3, 4]","[3, 4]","[{'arousal': 4.0, 'valence': 2.5, 'dominance':...","['Frustration', 'Fear', 'Excited']","3,5","3,2","3,4",Nervous
4,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M002,It must be them then.,011.4300-012.7900,Fear,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Fear']",Fear,"[4, 2]","[4, 3]","[3, 4]","[{'arousal': 3.0, 'valence': 2.5, 'dominance':...","['Fear', 'Fear', 'Excited']","2,4","2,3","3,4",Nervous


In [6]:
df = df[['Audio_Uttrance_Path', 'New_Category']]
df.head()

,Audio_Uttrance_Path,New_Category
0,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Nervous
1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Nervous
2,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Nervous
3,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Nervous
4,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Nervous


In [7]:
df['New_Category'].unique()


array(['Nervous', 'Neutral', 'Angry', 'Sad', 'Happy', 'Excited', nan],
      dtype=object)

In [8]:
df['New_Category'] = df['New_Category'].str.replace('Excited', 'Happy')

In [9]:
df = df[~df["New_Category"].str.contains('Nervous', na=False)]

In [10]:
df.dropna(subset=['New_Category'], inplace=True)

In [11]:
df['Audio_Uttrance_Path'] = df['Audio_Uttrance_Path'].str.replace('/content/drive/My Drive', '/content/drive/Shareddrives/CS298')

In [12]:
df

,Audio_Uttrance_Path,New_Category
5,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Neutral
6,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Neutral
7,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Neutral
8,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Neutral
10,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Angry
...,...,...
2554,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Sad
2555,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Sad
2556,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Sad
2557,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,Sad


In [13]:
df.rename({'New_Category': 'label'}, axis=1, inplace=True)

In [14]:
df.rename({'Audio_Uttrance_Path': 'path'}, axis=1, inplace=True)

In [15]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(test_df)

In [16]:


print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['path', 'label', '__index_level_0__'],
    num_rows: 620
})
Dataset({
    features: ['path', 'label', '__index_level_0__'],
    num_rows: 156
})


In [17]:
# We need to specify the input and output column
input_column = "path"
output_column = "label"

In [18]:
print(train_dataset[0])

{'path': '/content/drive/Shareddrives/CS298/CS 297/IEMOCAP/Session4/Audio_Uttrances/Ses04F_impro07/Ses04F_impro07_F020.wav', 'label': 'Happy', '__index_level_0__': 1293}


In [19]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 4 classes: ['Angry', 'Happy', 'Neutral', 'Sad']


In [20]:
from transformers import AutoConfig, Wav2Vec2Processor

In [21]:
model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

In [22]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The target sampling rate: 16000


In [23]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label
'''
def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result
    '''

'\ndef preprocess_function(examples):\n    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]\n    target_list = [label_to_id(label, label_list) for label in examples[output_column]]\n\n    result = processor(speech_list, sampling_rate=target_sampling_rate)\n    result["labels"] = list(target_list)\n\n    return result\n    '

In [24]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
def preprocess_function(examples):
    # Process each audio file path in the dataset
    speech_list = [speech_file_to_array_fn(path) for path in examples['path']]

    # Here, converting labels to indices if they are categorical
    target_list = [label_to_id(label, label_list) for label in examples['label']]

    # Example of further processing, such as normalizing audio lengths or features
    # For this example, we assume that each array should be padded or truncated to a fixed length
    padded_speech_list = pad_sequences(speech_list, maxlen=MAX_LENGTH, dtype='float32', padding='post', truncating='post', value=0.0)

    return {
        'input_values': padded_speech_list,
        'labels': np.array(target_list)
    }

# Applying the preprocessing function to the dataset (assuming the use of Hugging Face datasets library)


In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=16,  # Adjust the batch size according to your system's capabilities
    num_proc=1  # Using multiple processes to expedite the preprocessing
)

eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=16,
    num_proc=1
)


Map:   0%|          | 0/620 [00:00<?, ? examples/s]